<a href="https://colab.research.google.com/github/Hawkwellow/YCP_scrapping/blob/main/ycbscripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import json
import urllib
import shutil
import random
import glob
from urllib.request import Request, urlopen

In [3]:
shutil.rmtree("/content/models/")

FileNotFoundError: ignored

In [2]:


# Define an output folder
output_directory = os.path.join("models", "ycb")

# Define a list of objects to download from
# http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/
#objects_to_download = "all"
objects_to_download = ["001_chips_can"]
#objects_to_download = ["001_chips_can", 
#                        "002_master_chef_can",
#                       "003_cracker_box",
#                      "004_sugar_box"]

# You can edit this list to only download certain kinds of files.
# 'berkeley_rgbd' contains all of the depth maps and images from the Carmines.
# 'berkeley_rgb_highres' contains all of the high-res images from the Canon cameras.
# 'berkeley_processed' contains all of the segmented point clouds and textured meshes.
# 'google_16k' contains google meshes with 16k vertices.
# 'google_64k' contains google meshes with 64k vertices.
# 'google_512k' contains google meshes with 512k vertices.
# See the website for more details.
#files_to_download = ["berkeley_rgbd", "berkeley_rgb_highres", "berkeley_processed", "google_16k", "google_64k", "google_512k"]
files_to_download = ["berkeley_rgb_highres"]

# Extract all files from the downloaded .tgz, and remove .tgz files.
# If false, will just download all .tgz files to output_directory
extract = True

base_url = "http://ycb-benchmarks.s3-website-us-east-1.amazonaws.com/data/"
objects_url = "https://ycb-benchmarks.s3.amazonaws.com/data/objects.json"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)


def fetch_objects(url):
    """ Fetches the object information before download """
    response = urlopen(url)
    html = response.read()
    objects = json.loads(html)
    return objects["objects"]


def download_file(url, filename):
    """ Downloads files from a given URL """
    u = urlopen(url)
    f = open(filename,"wb")
    file_size = int(u.getheader("Content-Length"))    
    print("Downloading: {} ({} MB)".format(filename, file_size/1000000.0))

    file_size_dl = 0
    block_sz = 65536
    while True:
        buffer = u.read(block_sz)
        if not buffer:
            break

        file_size_dl += len(buffer)
        f.write(buffer)
        status = r"%10d  [%3.2f%%]" % (file_size_dl/1000000.0, file_size_dl * 100. / file_size)
        status = status + chr(8)*(len(status)+1)
        print(status)
    f.close()
    

def tgz_url(object, type):
    """ Get the TGZ file URL for a particular object and dataset type """
    if type in ["berkeley_rgbd", "berkeley_rgb_highres"]:
        return base_url + "berkeley/{object}/{object}_{type}.tgz".format(object=object,type=type)
    elif type in ["berkeley_processed"]:
        return base_url + "berkeley/{object}/{object}_berkeley_meshes.tgz".format(object=object,type=type)
    else:
        return base_url + "google/{object}_{type}.tgz".format(object=object,type=type)


def extract_tgz(filename, dir):
    """ Extract a TGZ file """
    tar_command = "tar -xzf {filename} -C {dir}".format(filename=filename,dir=dir)
    os.system(tar_command)
    os.remove(filename)

def check_url(url):
    """ Check the validity of a URL """
    try:
        request = Request(url)
        request.get_method = lambda : 'HEAD'
        response = urlopen(request)
        return True
    except Exception as e:
        return False


if __name__ == "__main__":

    # Grab all the object information
    objects = fetch_objects(objects_url)

    

    # Download each object for all objects and types specified
    for object in objects:
        if objects_to_download == "all" or object in objects_to_download:
            for file_type in files_to_download:
                url = tgz_url(object, file_type)


                if not check_url(url):
                    continue
                filename = "{path}/{object}_{file_type}.tgz".format(
                    path=output_directory,
                    object=object,
                    file_type=file_type)
                download_file(url, filename)
                if extract:
                    #画像ファイル(ex.N1_0.jpg)のパスとランダムに削除するためのリスト
                    #output_directory はycbまで
                    imagepath = output_directory + "/{object}".format(object = object)
                    imgfiles = glob.glob("/models/ycb/" + object + "/*.jpg")

                    extract_tgz(filename, output_directory)
                    shutil.rmtree(imagepath + "/poses") 
                    keep_list = random.sample(imgfiles, 100)

                    new_dir = output_directory + "/{object}_sampled".format(object = object)
                    #移動先のディレクトリを作成
                    os.makedirs(new_dir, exist_ok = True)
                    os.makedirs(new_dir + "/masks", exist_ok = True)
                    for tomove in keep_list:
                      #tomove:移動したいファイル　new_dir：移動先のディレクトリ
                      shutil.copy(tomove, new_dir)
                      #_sampledに移動したデータ(=keep_listに入ったデータ)に対応するmaskデータをmasksへとコピー
                      #keep_listの一つの要素のpath:
                      #'models/ycb/001_chips...N4_201.jpg'
                      #→'models/ycb/{objects}/masks/tomoveから抽出文字列.pbmにする。
                      path_and_id = tomove.rsplit('/',1)
                      mask_file = "/" + path_and_id[1].replace('.jpg','') + "_mask.pbm"

                      maskpath = imagepath + "/masks"
                      shutil.copy(maskpath + mask_file, new_dir + "/masks")

                shutil.rmtree(imagepath)          
                    

ストリーミング出力は最後の 5000 行に切り捨てられました。
      1286  [79.70%]
      1286  [79.71%]
      1286  [79.71%]
      1286  [79.71%]
      1286  [79.72%]
      1286  [79.72%]
      1286  [79.73%]
      1286  [79.73%]
      1286  [79.73%]
      1286  [79.74%]
      1286  [79.74%]
      1286  [79.75%]
      1286  [79.75%]
      1287  [79.76%]
      1287  [79.76%]
      1287  [79.76%]
      1287  [79.77%]
      1287  [79.77%]
      1287  [79.78%]
      1287  [79.78%]
      1287  [79.78%]
      1287  [79.79%]
      1287  [79.79%]
  

ValueError: ignored

In [ ]:
#100個残すデータを抽出、maskと各画像をコピーし新しいディレクトリに格納
object = "001_chips_can"
imagepath = output_directory + "/{object}/".format(object = object)
imgfiles = glob.glob(imagepath + "*.jpg")

keep_list = random.sample(imgfiles, 100)

new_dir = output_directory + "/{object}_sampled".format(object = object)
  #移動先のディレクトリを作成
os.makedirs(new_dir, exist_ok = True)
os.makedirs(new_dir + "/masks", exist_ok = True)
for tomove in keep_list:
  #tomove:移動したいファイル　new_dir：移動先のディレクトリ
  shutil.copy(tomove, new_dir)
  #_sampledに移動したデータ(=keep_listに入ったデータ)に対応するmaskデータをmasksへとコピー
  #keep_listの一つの要素のpath:
  #'models/ycb/001_chips...N4_201.jpg'
  #→'models/ycb/{objects}/masks/tomoveから抽出文字列.pbmにする。
  path_and_id = tomove.rsplit('/',1)
  mask_file = path_and_id[1].replace('.jpg','') + "_mask.pbm"

  maskpath = imagepath + "masks/"
  shutil.copy(maskpath + mask_file, new_dir + "/masks/")